In [ ]:
!mc cp s3/$VAULT_TOP_DIR/Accords/Construction_dataset_public/Dataset_public_accords_teletravail_Dares.parquet .

In [ ]:
!mc cp -r s3/$VAULT_TOP_DIR/Accords/chroma_db .

In [ ]:
!mc cp -r s3/$VAULT_TOP_DIR/Accords/Large2/results .

In [ ]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document, Generation, LLMResult
from langchain.llms import Ollama, BaseLLM
from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
import json
import requests

class LocalOllamaLLM(BaseLLM):
    api_url : str
    def _generate(self, prompt, stop):
        response = requests.post(f"{self.api_url}/api/generate", json={"model": "mistral-large", "prompt": str(prompt) })
        response.raise_for_status()
        response_text=''.join([json.loads(line)['response'] for line in response.text.splitlines()])
        generations=[]
        generations.append([Generation(text=response_text)])
        return LLMResult(generations=generations)


    def _llm_type(self):
        return "local"  # Or whatever type is appropriate for your local setup

llm = LocalOllamaLLM(api_url="http://127.0.0.1:11434")

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

system_prompt = (
    " Utilisez le contexte donné pour répondre à la question avec seulement 'Oui' ou 'Non'"
    " Contexte : {context}  "
)

system_prompt2 = (
    " Utilisez le contexte donné pour répondre à la question.  "
    " Si vous ne connaissez pas la réponse, dites que vous ne savez pas.  "
    " Utilisez trois phrases au maximum et soyez concis dans votre réponse. "
    " En premier lieu, répondre en donnant une variable : variable=(valeur ou None)  . "
    " S'il y a plusieurs valeurs possibles, prendre le max : variable=max(valeurs ou None)  . "
    " Contexte : {context}  "
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
file="Dataset_public_accords_teletravail_Dares.parquet"
df=pd.read_parquet(file)

In [ ]:
vector_store = Chroma(embedding_function=embedder,persist_directory="./chroma_db")

In [ ]:
def search_and_invoke_llm(vector_store,index,query,k=5):
    if k==0:
        print(f"bug with {index}")
    elif k==1:
        print(f" trying k=1 : {index}")
    else:
        pass
    try:
        retriever=vector_store.as_retriever(
        search_kwargs={
                "k": k, 
                "filter": {'index': index}
            }
        )
        chain = create_retrieval_chain(retriever, question_answer_chain)
        result=chain.invoke({"input": query})
        return result['answer']
    except:
        search_and_invoke_llm(vector_store,index,k=k-1)
    return None

In [ ]:
Q1="Est-ce qu'un nombre de jour de télétravail est mentionné ?"
Q2="Est-ce qu'une limite de jour de télétravail est mentionné ?"
Q3="Est-ce qu'une journée de télétravail est mentionné ?"
Q4="Est-ce qu'un nombre de jour de télétravail peut être déduit?"
Q5="Est-ce qu'une limite de jour de télétravail peut être déduit?"

LIST_OF_QUESTIONS=[Q1,Q2,Q3,Q4,Q5]

In [ ]:
from glob import glob
already_done={el.split("/")[1].split(".")[0] for el in glob("results/*.answer")}
new_dir = Path('results').mkdir(exist_ok=True)
#query= f"{Q1} {Q2} {Q3} {Q4} {Q5}"
for index, row in df.iterrows():
    answer=""
    if index not in already_done:

        for Q in LIST_OF_QUESTIONS:
            answer += search_and_invoke_llm(vector_store,index,Q)
        #query= "Combien de jour de télétravail par semaine est autorisé au maximum ?"
        #answer += search_and_invoke_llm(vector_store,index,query)
        print(index,df.nombre_jour_hebdo_TT_annotee[index], answer)
        if answer:
            with open(f"results/{index}.answer","w") as f:
                f.write(answer)

In [ ]:
#vector_store.similarity_search(Q4,2,{'index': "T59L21013979"})

In [ ]:
!mc cp -r results s3/$VAULT_TOP_DIR/Accords/

## Misc

In [ ]:
def iterate_in_chunks(lst, chunk_size=5):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]